In [2]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib  # For saving model, encoder, vectorizer (optional)

In [3]:
data = {
    'English': ['apple', 'banana', 'cat', 'dog', 'elephant', 'fish', 'goat', 'hat', 'ice', 'jug'],
    'Hindi': ['सेब', 'केला', 'बिल्ली', 'कुत्ता', 'हाथी', 'मछली', 'बकरी', 'टोपी', 'बर्फ़', 'जग']
}

df = pd.DataFrame(data)
df.to_csv("english_hindi.csv", index=False)
df = pd.read_csv("english_hindi.csv")
df.head()


,English,Hindi
0,apple,सेब
1,banana,केला
2,cat,बिल्ली
3,dog,कुत्ता
4,elephant,हाथी


In [4]:
X = df['English']
y = df['Hindi']

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert words to vector
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y_encoded, test_size=0.2, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Optional: Save models
joblib.dump(model, "model.pkl")
joblib.dump(le, "encoder.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")


Accuracy: 0.0
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       0.0
           5       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



C:\Users\kacha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kacha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kacha\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kacha\anaconda3\Lib\site-packages\sklearn\metrics\_cla

['vectorizer.pkl']

In [5]:
def translate():
    word = entry.get().lower()

    if word == "":
        messagebox.showerror("Input Error", "Please enter a word.")
        return

    # Check for vowel start
    if word[0] in ['a', 'e', 'i', 'o', 'u']:
        current_hour = datetime.now().hour
        if current_hour != 21:  # Only allow at 9 PM
            output_label.config(text="This word starts with a vowel. Please provide another word.")
            return

    try:
        word_vector = vectorizer.transform([word])
        prediction = model.predict(word_vector)
        hindi_word = le.inverse_transform(prediction)
        output_label.config(text=f"Hindi: {hindi_word[0]}")
    except Exception as e:
        output_label.config(text="Word not found in dataset.")

# GUI Design
root = tk.Tk()
root.title("English to Hindi Translator")
root.geometry("400x250")

tk.Label(root, text="Enter English Word:", font=('Arial', 12)).pack(pady=10)
entry = tk.Entry(root, font=('Arial', 14))
entry.pack()

tk.Button(root, text="Translate", command=translate, font=('Arial', 12)).pack(pady=10)

output_label = tk.Label(root, text="", font=('Arial', 14))
output_label.pack(pady=20)

root.mainloop()
